# Remote Sensing - Exploring Black Marble Imagery at Penn State

## Environment Setup

### Import Packages

In [ ]:
# Import packages
import os
import re
from collections import ChainMap
import warnings
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.transform import from_origin
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.mask as em
import radiance as rd

### Set Package Options

In [ ]:
# Set options
warnings.filterwarnings("ignore")
sns.set(font_scale=1.5)
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Set Working Directory

In [ ]:
# Set working directory (move up one folder)
os.chdir("..")

# Show working directory
os.getcwd()

## Data Preparation

### Define File Paths

In [ ]:
# Set path to radiance files
psu_lat_lon_path = os.path.join(
    "02-raw-data", "nasa-black-marble", 
    "penn-state-campus-sep-2018-may-2020-pixel-lat-lon.csv")

psu_radiance_path = os.path.join(
    "02-raw-data", "nasa-black-marble", 
    "penn-state-campus-sep-2018-may-2020-radiance.csv")

psu_cloud_mask_path = os.path.join(
    "02-raw-data", "nasa-black-marble", 
    "penn-state-campus-sep-2018-may-2020-cloud-mask.csv")

In [ ]:
# Check that all paths to files exist
for path in [psu_lat_lon_path, psu_radiance_path, psu_cloud_mask_path]:
    print(f"Path exists: {path}") if os.path.exists(
        path) else print(f"Path does not exist: {path}")

### Read CSV Files to Geodataframes

In [ ]:
# Read lat/lon into dataframe
psu_lat_lon_df = pd.read_csv(psu_lat_lon_path)

In [ ]:
# Display dataframe
psu_lat_lon_df.head()

In [ ]:
# Read radiance values to dataframe
psu_radiance_df = pd.read_csv(psu_radiance_path)

In [ ]:
psu_radiance_df.head()

In [ ]:
# Read cloud mask values to dataframe
psu_cloud_mask_df = pd.read_csv(psu_cloud_mask_path)

In [ ]:
psu_cloud_mask_df.head()

### Add Missings Dates and Data

In [ ]:
# Fill radiance data with missing dates (dates specified by user)
psu_radiance_filled = rd.add_missing_data(
    psu_radiance_df, '2018-09-01', '2020-05-07')

In [ ]:
# Display dataset with filled data (NaN)
psu_radiance_filled.head()

In [ ]:
# Fill cloud mask data with missing dates
psu_cloud_mask_filled = rd.add_missing_data(
    psu_cloud_mask_df, '2018-09-01', '2020-05-07')

In [ ]:
# Display filled dataset with filled data (NaN)
psu_cloud_mask_filled.head()

### Extract Plotting Extent & Transform

In [ ]:
# Extract PSU extent and transform from lat/lon dataframe
psu_extent, psu_transform, psu_shape = rd.extract_extent(
    study_area=psu_lat_lon_df, 
    longitude_column='Longitude', 
    latitude_column='Latitude')

In [ ]:
# Display extent (left, right, bottom, top)
psu_extent

In [ ]:
# Display tranform (used for export)
psu_transform

In [ ]:
# Display shape
psu_shape

### Create GeoDataFrame of Lat/Lon Points

In [ ]:
# Create geodataframe to store lat/lon points 
#  (for plotting over array)
psu_lat_lon_copy = psu_lat_lon_df.copy()

psu_lat_lon_gdf = gpd.GeoDataFrame(
    psu_lat_lon_copy, 
    crs="epsg:4326", 
    geometry=gpd.points_from_xy(
        psu_lat_lon_copy.Longitude, 
        psu_lat_lon_copy.Latitude)
)

In [ ]:
# Display dataframe
psu_lat_lon_gdf.head()

## Data Processing

**Penn State Academic Calendars**

[All Academic Calendars](http://www.registrar.psu.edu/academic-calendars/)

[2018-2019 Academic Calendar](http://www.registrar.psu.edu/academic-calendars/2018-19.cfm)

[2019-2020 Academic Calendar](http://www.registrar.psu.edu/academic-calendars/2019-20.cfm)

### Store Daily Radiance Values

In [ ]:
# Get list of dates (daily) from Sept 1, 2018 to May 7, 2020
date_list = rd.create_date_list(start_date='2018-09-01', end_date='2020-05-07')

In [ ]:
# Display number of days in list
len(date_list)

In [ ]:
# Store all daily filled values in nested dictionary
#  (indexed by [Year][Month][Day])
radiance_sept_2018_may_2020 = rd.store_data(
    psu_radiance_filled, psu_cloud_mask_filled, 
    mask_value=100, array_shape=psu_shape, dates=date_list)

In [ ]:
# Initialize counters for years/months/days in dictionary
years = months = days = 0
radiance_dict = radiance_sept_2018_may_2020

# Loop through all years
for key_year in radiance_dict.keys():

    # Add to counter for each year
    years += 1
    
    # Loop through all months
    for key_month in radiance_dict.get(key_year).keys():
        
        # Add to counter for each month
        months += 1
        
        # Loop through all days
        for key_day in radiance_dict.get(key_year).get(key_month):
        
            # Add to counter for each day
            days += 1

In [ ]:
# Display number of years/months/days in dictionary
print(f"Entries in Radiance Dictionary\nYears:    {years}\nMonths:  {months}\nDays:   {days}")

### Date Range Radiance - Monthly Mean Radiance

In [ ]:
# Define year/month combinations to loop through
calendar_months_2018_2020 = [
    '2018-09', '2018-10', '2018-11', '2018-12',
    '2019-01', '2019-02', '2019-03', '2019-04',
    '2019-05', '2019-06', '2019-07', '2019-08',
    '2019-09', '2019-10', '2019-11', '2019-12',
    '2020-01', '2020-02', '2020-03', '2020-04'
]

In [ ]:
# Store monthly means in dictionary
radiance_monthtly_mean = rd.store_monthly_mean(
    radiance_daily=radiance_sept_2018_may_2020, 
    dates=calendar_months_2018_2020)

In [ ]:
# Show top-level keys (years)
radiance_monthtly_mean.keys()

In [ ]:
# Show 2018 keys (months)
radiance_monthtly_mean.get('2018').keys()

In [ ]:
# Show 2019 keys (months)
radiance_monthtly_mean.get('2019').keys()

In [ ]:
# Show 2020 keys (months)
radiance_monthtly_mean.get('2020').keys()

In [ ]:
# Get min and max values for September 2019 mean
print(f"Min Radiance, Sept 2019 Mean Values: {radiance_monthtly_mean.get('2019').get('09').min()}")
print(f"Max Radiance, Sept 2019 Mean Values: {radiance_monthtly_mean.get('2019').get('09').max()}")

In [ ]:
# Get min and max values for March 2020 mean
print(f"Min Radiance, March 2020 Mean Values: {radiance_monthtly_mean.get('2020').get('03').min()}")
print(f"Max Radiance, March 2020 Mean Values: {radiance_monthtly_mean.get('2020').get('03').max()}")

### Date Range Radiance Mean - Continuous Date Ranges

In [ ]:
# Define date ranges (start date, end date) to loop through
academic_years_2018_2020 = [

    # Labor day weekend to commencement weekend (includes day after commencement)
    ('2018-09-01', '2018-12-16'),

    # Thanksgiving holiday
    ('2018-11-18', '2018-11-24'),

    # Fall study days & final exam week
    ('2018-12-08', '2018-12-14'),

    # After Fall commencement to before Spring new student arrival
    ('2018-12-17', '2019-01-04'),

    # Spring new student arrival to Spring commencement (commencement weekend)
    ('2019-01-05', '2019-05-05'),

    # Spring break
    ('2019-03-03', '2019-03-09'),

    # Spring study days & final exam week
    ('2019-04-27', '2019-05-03'),

    # Maymester (excluding arrival day, which occurs on Spring commencement)
    ('2019-05-06', '2019-06-05'),

    # Summer session (regular) through commencement (includes day after commencement)
    ('2019-05-12', '2019-08-11'),


    # 2019-2020 academic year
    # Fall new student arrival to commencement weekend (includes day after commencement)
    ('2019-08-23', '2019-12-22'),

    # Labor day weekend to Commencement weekend (includes day after commencement)
    ('2019-08-31', '2019-12-22'),

    # Thanksgiving holiday
    ('2019-11-24', '2019-11-30'),

    # Fall study days & final exam week
    ('2019-12-14', '2019-12-20'),

    # After Fall commencement to before Spring new student arrival
    ('2019-12-23', '2020-01-10'),

    # Spring new student arrival to Spring commencement (commencement weekend)
    ('2020-01-11', '2020-05-10'),

    # Spring break
    ('2020-03-08', '2020-03-14'),

    # Spring study days & final exam week
    ('2020-05-02', '2020-05-08')

    #     # Maymester (excluding arrival day, which occurs on Spring commencement)
    #     rd.create_date_list('2020-05-11', '2020-06-10'),

    #     # Summer session (regular) through commencement (includes day after commencement)
    #     rd.create_date_list('2020-05-18', '2020-08-15')
]

In [ ]:
# Store continuous ranges means in dictionary
academic_years_2018_2020_means = rd.store_continuous_range_mean(
    radiance_daily=radiance_sept_2018_may_2020,
    date_range_list=academic_years_2018_2020)

In [ ]:
# Show all keys in dictionary
for key in academic_years_2018_2020_means.keys():
    print(key)

In [ ]:
# Display number of keys
len(academic_years_2018_2020_means.keys())

In [ ]:
# Display shape of single mean array in dictionary
academic_years_2018_2020_means.get('20190512-20190811').shape

### Date Range Radiance - Recurring Weekly Day Means (SUN, MON, TUE, etc.)

In [ ]:
# Define date ranges for academic semesters
semester_ranges = [
    
    # Fall 2018
    ('2018-09-01', '2018-12-16'),
    
    # Spring 2019
    ('2019-01-05', '2019-05-05'),

    # Summer 2019
    ('2019-05-12', '2019-08-11'),
    
    # Fall 2019
    ('2019-08-31', '2019-12-22'),
    
    # Spring 2020
    ('2020-01-11', '2020-05-10')
]

In [ ]:
# Create list of dictionaries with stored semester weekly radiances
semesters_weekly = [rd.store_weekly_range_mean(radiance_sept_2018_may_2020, start_date, end_date)
                    for start_date, end_date in semester_ranges]

In [ ]:
# Unpack list of dictionaries to single dictionary
semesters_unpacked = rd.unpack_dictionaries(semesters_weekly)

# Display all keys in unpacked dictionary
for key in semesters_unpacked.keys():
    print(key)

In [ ]:
# Display number of keys
len(semesters_unpacked.keys())

In [ ]:
# Display shape of single mean array in dictionary
semesters_unpacked.get('20190831-20191222-MON').shape

## Data Visualization

### Create Ouput Directories

In [ ]:
import os

# Define paths to output directories
processed_nasa_dir = os.path.join(
    "03-processed-data", "nasa-black-marble")

graphics_nasa_dir = os.path.join(
    "04-graphics-outputs", "nasa-black-marble")

nasa_geo_dir = os.path.join(
    graphics_nasa_dir, "georeferenced")

nasa_nongeo_dir = nasa_geo_dir = os.path.join(
    graphics_nasa_dir, "non-georeferenced")

# Create list of directories
dir_list = [
    processed_nasa_dir, graphics_nasa_dir, 
    geo_dir, nongeo_dir
]

# Create non-existing directories
for dir in dir_list:
    if not os.path.exists(dir):
        os.mkdir(dir)
        print(f"Created directory: {dir}")
    else:
        print(f"Already exists: {dir}")

### Plot and Save Figures to PNG

In [ ]:
# Plot Sept 2019 mean
fig, ax = rd.plot_values(
    radiance_monthtly_mean.get('2019').get('09'),
    title="September 2019 Mean Radiance",
    difference=False)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-mean-sep-2019-values.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Plot Sept 2019 histogram
fig, ax = rd.plot_histogram(
    radiance_monthtly_mean.get('2019').get('09'), 
    title="Distribution of September 2019 Mean Radiance",
    xlabel='Mean Radiance',
    difference=False)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-mean-sep-2019-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Plot March 2020 mean
fig, ax = rd.plot_values(
    radiance_monthtly_mean.get('2020').get('03'), 
    title="March 2020 Mean Radiance",
    difference=False)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-mean-mar-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Plot March 2020 histogram
fig, ax = rd.plot_histogram(
    radiance_monthtly_mean.get('2020').get('03'), 
    title="Distribution of March 2020 Mean Radiance",
    xlabel='Mean Radiance',
    difference=False)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-mean-mar-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate change, September 2019 mean to March 2020 mean
diff_sep_2019_march_2020 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2020').get('03'),
    radiance_monthtly_mean.get('2019').get('09'))

# Plot difference from Sept 2019 to March 2020
fig, ax = rd.plot_values(
    diff_sep_2019_march_2020, 
    title="Change in Mean Radiance (September 2019 vs. March 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-sep-2019-mar-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Plot Sept 2019 vs. March 2020 change histogram
fig, ax = rd.plot_histogram(
    diff_sep_2019_march_2020, 
    title="Distribution of the Change in Mean Radiance (September 2019 vs. March 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-sep-2019-mar-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

### Plot and Save Figures to PNG - Monthly Mean Difference, 1 Year Gap

In [ ]:
# Calculate September 2018 to September 2019 difference
diff_sep_2018_sep_2019 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2019').get('09'),
    radiance_monthtly_mean.get('2018').get('09'))

# Plot difference
fig, ax = rd.plot_values(
    diff_sep_2018_sep_2019, 
    title="Change in Mean Radiance (September 2018 vs. September 2019)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-sep-2018-sep-2019-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_sep_2018_sep_2019, 
    title="Distribution of the Change in Mean Radiance (September 2018 vs. September 2019)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-sep-2018-sep-2019-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate October 2018 to October 2019 difference
diff_oct_2018_oct_2019 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2019').get('10'),
    radiance_monthtly_mean.get('2018').get('10'))

# Plot difference
fig, ax = rd.plot_values(
    diff_oct_2018_oct_2019, 
    title="Change in Mean Radiance (October 2018 vs. October 2019)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-oct-2018-oct-2019-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_oct_2018_oct_2019, 
    title="Distribution of the Change in Mean Radiance (October 2018 vs. October 2019)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-oct-2018-oct-2019-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate November 2018 to November 2019 difference
diff_nov_2018_nov_2019 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2019').get('11'),
    radiance_monthtly_mean.get('2018').get('11'))

# Plot difference
fig, ax = rd.plot_values(
    diff_nov_2018_nov_2019, 
    title="Change in Mean Radiance (November 2018 vs. November 2019)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-nov-2018-nov-2019-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_nov_2018_nov_2019, 
    title="Distribution of the Change in Mean Radiance (November 2018 vs. November 2019)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-nov-2018-nov-2019-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate December 2018 to December 2019 difference
diff_dec_2018_dec_2019 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2019').get('12'),
    radiance_monthtly_mean.get('2018').get('12'))

# Plot difference
fig, ax = rd.plot_values(
    diff_dec_2018_dec_2019, 
    title="Change in Mean Radiance (December 2018 vs. December 2019)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced", 
    "penn-state-campus-radiance-difference-dec-2018-dec-2019-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_dec_2018_dec_2019, 
    title="Distribution of the Change in Mean Radiance (December 2018 vs. December 2019)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-dec-2018-dec-2019-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate January 2019 to January 2020 difference
diff_jan_2019_jan_2020 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2020').get('01'),
    radiance_monthtly_mean.get('2019').get('01'))

# Plot difference
fig, ax = rd.plot_values(
    diff_jan_2019_jan_2020, 
    title="Change in Mean Radiance (January 2019 vs. January 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-jan-2019-jan-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_jan_2019_jan_2020, 
    title="Distribution of the Change in Mean Radiance (January 2019 vs. January 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-jan-2019-jan-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate February 2019 to February 2020 difference
diff_feb_2019_feb_2020 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2020').get('02'),
    radiance_monthtly_mean.get('2019').get('02'))

# Plot difference
fig, ax = rd.plot_values(
    diff_feb_2019_feb_2020, 
    title="Change in Mean Radiance (February 2019 vs. February 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-feb-2019-feb-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_feb_2019_feb_2020, 
    title="Distribution of the Change in Mean Radiance (February 2019 vs. February 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-feb-2019-feb-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate March 2019 to March 2020 difference
diff_march_2019_march_2020 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2020').get('03'),
    radiance_monthtly_mean.get('2019').get('03'))

# Plot difference
fig, ax = rd.plot_values(
    diff_march_2019_march_2020, 
    title="Change in Mean Radiance (March 2019 vs. March 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced", 
    "penn-state-campus-radiance-difference-mar-2019-mar-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_march_2019_march_2020, 
    title="Distribution of the Change in Mean Radiance (March 2019 vs. March 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-mar-2019-mar-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Calculate April 2019 to April 2020 difference
diff_april_2019_april_2020 = rd.subtract_arrays(
    radiance_monthtly_mean.get('2020').get('04'),
    radiance_monthtly_mean.get('2019').get('04'))

# Plot difference
fig, ax = rd.plot_values(
    diff_april_2019_april_2020, 
    title="Change in Mean Radiance (April 2019 vs. April 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced", 
    "penn-state-campus-radiance-difference-apr-2019-apr-2020-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_april_2019_april_2020, 
    title="Distribution of the Change in Mean Radiance (April 2019 vs. April 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-apr-2019-apr-2020-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

### Plot and Save Figures to PNG - Weekly Day Mean Difference, 1 Year Gap, Spring Semester

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Sunday means
diff_spring_2019_2020_sundays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-SUN'),
    semesters_unpacked.get('20190105-20190505-SUN'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_sundays, 
    title="Change in Mean Radiance (Sunday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-sun-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_sundays, 
    title="Distribution of the Change in Mean Radiance (Sunday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-sun-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Monday means
diff_spring_2019_2020_mondays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-MON'),
    semesters_unpacked.get('20190105-20190505-MON'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_mondays, 
    title="Change in Mean Radiance (Monday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-mon-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_mondays, 
    title="Distribution of the Change in Mean Radiance (Monday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-mon-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Tuesday means
diff_spring_2019_2020_tuesdays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-TUE'),
    semesters_unpacked.get('20190105-20190505-TUE'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_tuesdays, 
    title="Change in Mean Radiance (Tuesday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-tue-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_tuesdays, 
    title="Distribution of the Change in Mean Radiance (Tuesday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-tue-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Wednesday means
diff_spring_2019_2020_wednesdays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-WED'),
    semesters_unpacked.get('20190105-20190505-WED'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_wednesdays, 
    title="Change in Mean Radiance (Wednesday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-wed-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_wednesdays, 
    title="Distribution of the Change in Mean Radiance (Wednesday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-wed-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Thursday means
diff_spring_2019_2020_thursdays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-THU'),
    semesters_unpacked.get('20190105-20190505-THU'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_thursdays, 
    title="Change in Mean Radiance (Thursday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-thu-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_thursdays, 
    title="Distribution of the Change in Mean Radiance (Thursday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced", 
    "penn-state-campus-radiance-difference-spring-2019-2020-thu-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Friday means
diff_spring_2019_2020_fridays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-FRI'),
    semesters_unpacked.get('20190105-20190505-FRI'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_fridays, 
    title="Change in Mean Radiance (Friday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-fri-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_fridays, 
    title="Distribution of the Change in Mean Radiance (Friday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-fri-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Saturday means
diff_spring_2019_2020_saturdays = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-SAT'),
    semesters_unpacked.get('20190105-20190505-SAT'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_saturdays, 
    title="Change in Mean Radiance (Saturday - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-sat-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_saturdays, 
    title="Distribution of the Change in Mean Radiance (Saturday - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-sat-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

In [ ]:
# Get difference between Spring 2019 and Spring 2020 Business Day means
diff_spring_2019_2020_business = rd.subtract_arrays(
    semesters_unpacked.get('20200111-20200510-BUS'),
    semesters_unpacked.get('20190105-20190505-BUS'))

# Plot difference
fig, ax = rd.plot_values(
    diff_spring_2019_2020_business, 
    title="Change in Mean Radiance (Business Days - Spring 2019 vs. Spring 2020)",
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-spring-2019-2020-bus-values.png")

# Save figure as PNG
rd.save_figure(outpath)

# Plot histogram
fig, ax = rd.plot_histogram(
    diff_spring_2019_2020_business, 
    title="Distribution of the Change in Mean Radiance (Business Days - Spring 2019 vs. Spring 2020)",
    xlabel='Change in Mean Radiance',
    difference=True)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced", 
    "penn-state-campus-radiance-difference-spring-2019-2020-bus-histogram.png")

# Save figure as PNG
rd.save_figure(outpath)

### Combined Plots

In [ ]:
# Define titles
plot_titles = [
    'September 2019 Mean Radiance',
    'March 2020 Mean Radiance',
    'Change in Mean Radiance (September 2019 vs. March 2020)'
]

# Plot Sept 2019 and March 2020
fig, ax = rd.plot_change(
    pre_change=radiance_monthtly_mean.get('2019').get('09'),
    post_change=radiance_monthtly_mean.get('2020').get('03'),
    titles=plot_titles)

# Define output path
outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "non-georeferenced",
    "penn-state-campus-radiance-difference-sep-2019-mar-2020-values-all.png")

# Save figure as PNG
rd.save_figure(outpath)

## Data Export

### Export Filled Dataframes to CSV

In [ ]:
# Set index column name
psu_cloud_mask_filled.index.name = 'pixel_id'

# Set output path
df_out_path = os.path.join(
    "03-processed-data", "nasa-black-marble",
    "penn-state-campus-sep-2018-may-2020-cloud-mask-filled.csv")

# Write data to CSV
try:
    psu_cloud_mask_filled.to_csv(
        path_or_buf=df_out_path, sep=',', header=True, index=True, na_rep=np.nan)
    
except Exception as error:
    print(f"Could not write to CSV. ERROR: {error}")
    
else:
    print(f"Wrote data to CSV: {os.path.split(df_out_path)[-1]}")

In [ ]:
# Set index column name
psu_radiance_filled.index.name = 'pixel_id'

# Set output path
df_out_path = os.path.join(
    "03-processed-data", "nasa-black-marble",
    "penn-state-campus-sep-2018-may-2020-radiance-filled.csv")

# Write data to CSV
try:
    psu_radiance_filled.to_csv(
        path_or_buf=df_out_path, sep=',', header=True, index=True, na_rep=np.nan)

except Exception as error:
    print(f"Could not write to CSV. ERROR: {error}")

else:
    print(f"Wrote data to CSV: {os.path.split(df_out_path)[-1]}")

### Export Radiance Arrays to GeoTiff

In [ ]:
# Define export output paths
sept_2019_radiance_outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "georeferenced",
    "penn-state-campus-radiance-sept-2019.tif")

march_2020_radiance_outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "georeferenced",
    "penn-state-campus-radiance-march-2020.tif")

radiance_diff_outpath = os.path.join(
    "04-graphics-outputs", "nasa-black-marble", "georeferenced",
    "penn-state-campus-radiance-difference-sept-2019-march-2020.tif")

In [ ]:
# Define export metadata
psu_campus_metadata = rd.create_metadata(
    array=diff_sep_2019_march_2020, transform=psu_transform)

In [ ]:
# Export September 2019 radiance
rd.export_array(
    array=radiance_monthtly_mean.get('2019').get('09'),
    output_path=sept_2019_radiance_outpath,
    metadata=psu_campus_metadata)

# Export March 2020 radiance
rd.export_array(
    array=radiance_monthtly_mean.get('2020').get('03'),
    output_path=march_2020_radiance_outpath,
    metadata=psu_campus_metadata)

# Export radiance difference
rd.export_array(
    array=diff_sep_2019_march_2020,
    output_path=radiance_diff_outpath,
    metadata=psu_campus_metadata)